In [2]:
%load_ext autoreload
%autoreload 2
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"
import torch  

from curious.utils.utils import move_paddings_to_right
from curious.config import TrainingConfig, RLConfig, WandbConfig, BaseConfig, SamplingConfig, RewardConfig, SFLConfig 
from curious.train.training_setup import set_up_training
from curious.train.trainer import PolicyGradientTrainer

PAD_TOKEN_ID = 128

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Utils 
*** 

In [3]:
attention_mask = torch.tensor(
    [
        [0, 0, 1, 1, 1],
        [0, 1, 1, 1, 1],
    ]
)

input_ids = torch.tensor(
    [
        [PAD_TOKEN_ID, PAD_TOKEN_ID, 1, 2, 3,],
        [PAD_TOKEN_ID, 1,            2, 3, 4,],
    ]
)


sequence_ids = torch.tensor(
    [
        [PAD_TOKEN_ID, PAD_TOKEN_ID, 1, 2, 3, 890, 891, PAD_TOKEN_ID, PAD_TOKEN_ID, PAD_TOKEN_ID],
        [PAD_TOKEN_ID, 1,            2, 3, 4, 899, 900, 911,          PAD_TOKEN_ID, PAD_TOKEN_ID],
    ]
)

move_paddings_to_right(input_ids, attention_mask, sequence_ids, PAD_TOKEN_ID)

3 10 7


(tensor([[  1,   2,   3, 890, 891, 128, 128],
         [  1,   2,   3,   4, 899, 900, 911]]),
 tensor([[False, False,  True,  True, False, False],
         [False, False, False,  True,  True,  True]]))

# Loading 
*** 

In [4]:
train_config = TrainingConfig(
    rl_config=RLConfig(
        group_size=3, 
        mini_batch_size=6,
        
    ),
    wandb_config=WandbConfig(),
    base_config=BaseConfig(
        train_batch_size=2,
        
    ),
    sampling_config=SamplingConfig(
        max_new_tokens=256,
    ),
    reward_config=RewardConfig(),
    sfl_config=SFLConfig(
    ),
)
#train_config.rl_config
training_setup, init_train_state = set_up_training(train_config)

#### Loading target policy Qwen/Qwen2-0.5B-Instruct ####
Applied Liger kernels to Qwen2
#### Loading dataset openai/gsm8k ####
Detected train_max_length: 212
Setting train_max_length to 212
Detected test_max_length: 188
Setting test_max_length to 188
#### Loading rollout data loader ####
#### Loading KL controller ####
#### Loading reference model ####
Applied Liger kernels to Qwen2
#### Defining actor loss ####
#### Defining reward model ####
#### Defining generation config ####
#### Defining evaluation config ####
#### Defining optimizer ####
#### Defining lr scheduler ####


In [5]:
init_train_state.keys()

dict_keys(['run_name', 'device', 'seed', 'model', 'optimizer', 'lr_scheduler', 'reference_model', 'kl_controller'])

In [6]:
trainer = PolicyGradientTrainer(training_setup) 

In [7]:
next_state, next_replay_buffer = trainer.train(init_train_state)   

Epoch 0 of 1

82 436 354


W0510 15:30:16.691000 279638 /system/conda/miniconda3/envs/cloudspace/lib/python3.11/site-packages/torch/_dynamo/variables/tensor.py:869] [1/0] Graph break from `Tensor.item()`, consider setting:
W0510 15:30:16.691000 279638 /system/conda/miniconda3/envs/cloudspace/lib/python3.11/site-packages/torch/_dynamo/variables/tensor.py:869] [1/0]     torch._dynamo.config.capture_scalar_outputs = True
W0510 15:30:16.691000 279638 /system/conda/miniconda3/envs/cloudspace/lib/python3.11/site-packages/torch/_dynamo/variables/tensor.py:869] [1/0] or:
W0510 15:30:16.691000 279638 /system/conda/miniconda3/envs/cloudspace/lib/python3.11/site-packages/torch/_dynamo/variables/tensor.py:869] [1/0]     env TORCHDYNAMO_CAPTURE_SCALAR_OUTPUTS=1
W0510 15:30:16.691000 279638 /system/conda/miniconda3/envs/cloudspace/lib/python3.11/site-packages/torch/_dynamo/variables/tensor.py:869] [1/0] to include these operations in the captured graph.
W0510 15:30:16.691000 279638 /system/conda/miniconda3/envs/cloudspace/lib

Batch indx 0

torch.cuda.memory_allocated: 1.872596GB

torch.cuda.memory_reserved: 2.974609GB

torch.cuda.max_memory_reserved: 3.009766GB

{
    'train/mean_batch_returns': -0.416015625,
    'train/mean_batch_solved_rate': 0.1669921875,
    'train/mean_num_words_in_completions': 38.0,
    'train/max_num_words_in_completions': 82.0,
    'train/min_num_words_in_completions': 1.0,
    'train/mean_batch_format_returns': 0.0,
    'train/mean_batch_outcome_returns': -0.4166666666666667,
    'train/mean_batch_length_penalty': 0.0,
    'train/lr': 1e-06,
    'train/mean_action_entropy': 0.28125,
    'num_batches_visited': 0
}

6

In [8]:
exp = next_replay_buffer[0]

In [9]:
tokenizer = trainer.tokenizer

In [10]:
for key in exp.keys:
    val = getattr(exp, key)
    if val is not None:
        print(key)
        print(val.shape)

sequences
torch.Size([354])
action_log_probs
torch.Size([353])
log_probs_ref
torch.Size([353])
returns
torch.Size([])
solved_mask
torch.Size([])
advantages
torch.Size([])
attention_mask
torch.Size([354])
action_mask
torch.Size([353])


In [11]:
print(tokenizer.decode(exp.sequences))

<|im_start|>system
Please reason step by step, and put your final answer within $\boxed{}$.<|im_end|>
<|im_start|>user
Elizabeth has 20 dollars and wants to buy pens and pencils. Each pencil costs $1.60 and each pen cost 2 dollars. How many pencils can she buy with her 20 dollars if she wants 6 pens?<|im_end|>
<|im_start|>assistant
Elizabeth wants to buy 6 pens, so she has 6 * $2 = $12 to spend.
She has $20 and she has $12 to spend, so she can buy $20 - $12 = $8 more pencils.
Each pencil costs $1.60, so with $8, she can buy $8 / $1.60 = 5 pencils.
The answer is: $\boxed{5}$<|im_end|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endof

In [12]:
print(
    tokenizer.decode(
        exp.sequences[exp.attention_mask]
    )
)

<|im_start|>system
Please reason step by step, and put your final answer within $\boxed{}$.<|im_end|>
<|im_start|>user
Elizabeth has 20 dollars and wants to buy pens and pencils. Each pencil costs $1.60 and each pen cost 2 dollars. How many pencils can she buy with her 20 dollars if she wants 6 pens?<|im_end|>
<|im_start|>assistant
Elizabeth wants to buy 6 pens, so she has 6 * $2 = $12 to spend.
She has $20 and she has $12 to spend, so she can buy $20 - $12 = $8 more pencils.
Each pencil costs $1.60, so with $8, she can buy $8 / $1.60 = 5 pencils.
The answer is: $\boxed{5}$<|im_end|>


In [13]:
print(exp.returns)

tensor(-1., dtype=torch.bfloat16)


In [14]:
print(exp.solved_mask)

tensor(0., dtype=torch.bfloat16)


In [15]:
print(exp.advantages)

tensor(-0.5781, dtype=torch.bfloat16)


In [16]:
exp1 = next_replay_buffer[1]
print(exp1.solved_mask)
print(exp1.returns)
print(exp1.advantages)
print(
    tokenizer.decode(
        exp1.sequences[exp1.attention_mask]
    )
)

tensor(1., dtype=torch.bfloat16)
tensor(1., dtype=torch.bfloat16)
tensor(1.1562, dtype=torch.bfloat16)
<|im_start|>system
Please reason step by step, and put your final answer within $\boxed{}$.<|im_end|>
<|im_start|>user
Elizabeth has 20 dollars and wants to buy pens and pencils. Each pencil costs $1.60 and each pen cost 2 dollars. How many pencils can she buy with her 20 dollars if she wants 6 pens?<|im_end|>
<|im_start|>assistant
 Elizabeth wants to buy 6 pens, and each pen costs 2 dollars, so the total cost of the pens is 6 * 2 = 12 dollars.
Elizabeth has 20 dollars in total, and she wants to buy 12 dollars worth of pens. This means she has 20 - 12 = 8 dollars left to spend on pencils.
Each pencil costs 1.60 dollars, so to find out how many pencils she can buy with the remaining money, we divide the remaining money by the cost of each pencil: 8 / 1.60 = 5.
So, Elizabeth can buy 5 pencils with her remaining 8 dollars.
The answer is: $\boxed{5}$<|im_end|>


In [17]:
exp2 = next_replay_buffer[2]
print(exp2.solved_mask)
print(exp2.returns)
print(exp2.advantages)
print(
    tokenizer.decode(
        exp2.sequences[exp2.attention_mask]
    )
)

tensor(0., dtype=torch.bfloat16)
tensor(-1., dtype=torch.bfloat16)
tensor(-0.5781, dtype=torch.bfloat16)
<|im_start|>system
Please reason step by step, and put your final answer within $\boxed{}$.<|im_end|>
<|im_start|>user
Elizabeth has 20 dollars and wants to buy pens and pencils. Each pencil costs $1.60 and each pen cost 2 dollars. How many pencils can she buy with her 20 dollars if she wants 6 pens?<|im_end|>
<|im_start|>assistant
 Elizabeth wants to buy 6 pens, and each pen costs 2 dollars, so the total cost of the pens is 6 * 2 = 12 dollars.
Elizabeth has 20 dollars in total, so she can spend 20 - 12 = 8 dollars on pencils.
Each pencil costs 1.60 dollars, so to find out how many pencils she can buy, we divide the amount of money she has left by the cost of each pencil: 8 / 1.60 = 5.
Elizabeth can buy 5 pencils with her 20 dollars.
The answer is: $\boxed{5}$<|im_end|>


In [18]:
exp3 = next_replay_buffer[3]
print(exp3.solved_mask)
print(exp3.returns)
print(exp3.advantages)
print(
    tokenizer.decode(
        exp3.sequences[exp3.attention_mask]
    )
)

tensor(0., dtype=torch.bfloat16)
tensor(-0.5000, dtype=torch.bfloat16)
tensor(0., dtype=torch.bfloat16)
<|im_start|>system
Please reason step by step, and put your final answer within $\boxed{}$.<|im_end|>
<|im_start|>user
Billy and Tiffany are having a contest to see how can run the most miles in a week. On Sunday, Monday, and Tuesday, Billy runs 1 mile each day and Tiffany runs 2 miles each day. On Wednesday, Thursday, and Friday, Billy runs 1 mile each day and Tiffany runs a 1/3 of a mile each day. On Saturday Tiffany assumes she's going to win and takes the day off. How many miles does Billy have to run on Saturday to tie Tiffany?<|im_end|>
<|im_start|>assistant
 On Sunday, Monday, and Tuesday, Billy runs 1 mile each day, so he runs a total of 1+1+1 = 3 miles.
On Wednesday, Thursday, and Friday, Billy runs 1 mile each day, so he runs a total of 1+1+1 = 3 miles.
On Saturday, Tiffany takes the day off, so she doesn't run any miles.
Tiffany runs a 1/3 of a mile each day from Sunday to

In [19]:
print(
    tokenizer.decode(
        exp3.sequences[1:][exp3.action_mask]
    )
)

 On Sunday, Monday, and Tuesday, Billy runs 1 mile each day, so he runs a total of 1+1+1 = 3 miles.
On Wednesday, Thursday, and Friday, Billy runs 1 mile each day, so he runs a total of 1+1+1 = 3 miles.
On Saturday, Tiffany takes the day off, so she doesn't run any miles.
Tiffany runs a 1/3 of a mile each day from Sunday to Friday, so she runs 1/3*5 = 1.67 miles in total.
Tiffany runs 1.67 miles in a week, and Billy runs 3 miles, so Billy needs to run 3-1.67 = 1.33 miles on Saturday to tie Tiffany.
However, Billy can only run whole miles, so he needs to run an additional 1 mile on Saturday.
Billy needs to run 1.33+1 = 2.33 miles on Saturday to tie Tiffany.
The answer is: $\boxed{2.33}$<|im_end|>
